<a href="https://colab.research.google.com/github/coldbilberry/repo-gui/blob/main/ML_in_Business_HW7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.metrics import (mean_absolute_error, r2_score, recall_score, precision_score,
                            roc_auc_score, accuracy_score, f1_score)
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import xgboost as xgb

import shap
# load JS visualization code to notebook
shap.initjs()

In [ ]:
# dataset из задания к уроку 6 файл "arrhythmia.data"
df = pd.read_csv("arrhythmia.data", header=None)
df.head(3)

In [ ]:
data = df.copy()
data[279].replace([1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 0, inplace=True)
data[279].replace(2, 1, inplace=True)
data[279]

In [ ]:
data.iloc[:, -1].value_counts()

In [ ]:
def get_num_features(df):
    """
    функция возвращает список числовых признаков
    :param df
    :return: list
    """
    numerical_features = df.select_dtypes(include=[np.number])
    # print(f"count of numeric_features_train {numerical_features.shape[1]}")
    return numerical_features.columns


def get_cat_features(df):
    """
    функция возвращает список признаков с типом "объект"
    :param df:
    :return: list
    """
    categorical_features = df.select_dtypes(include=[np.object])
    return categorical_features.columns

In [ ]:
get_num_features(data)

In [ ]:
get_cat_features(data)

In [ ]:
sns.displot(data, x=data[279]);

In [ ]:
df = pd.DataFrame({
    # среднее по всем фичам трейна за исключением таргета
    'train mean 1': data.loc[data[279]==1, :].describe().iloc[1,1:],
    # среднее по всем фичам теста (здесь таргета нет)
    'train mean 0': data.loc[data[279]==0, :].describe().iloc[1,1:]
})

df.plot.bar(figsize=(16,8))
plt.title('Среднее значение целевой переменной для разных классов датасета');

In [ ]:
df =  data.loc[data[279]==1, :].describe().iloc[1,1:] - data.loc[data[279]==0, :].describe().iloc[1,1:]

df.plot(figsize=(16,8))
plt.title('Среднее значение целевой переменной');

In [ ]:
data[10].unique()

In [ ]:
for col in [10, 11, 12, 13, 14]:
    # извлекаем индексы по условию (тех элементов, которые имеют значение "?")
    indices = np.where(data[col] != '?')
    # получаем спиcок элементов по индексам [0], приводим к строке, к целому и находим медиану
    col_med = np.median(np.take(data[col], indices[0]).astype(str).astype(int))
    # заменяем некорректное значение медианой
    data[col] = data[col].replace('?', col_med)

In [ ]:
data[10].unique()

In [ ]:
for col in [10, 11, 12, 13, 14]:
    data[col] = data[col].astype(int)

In [ ]:
data[10].unique()

In [ ]:
%%time
parameters = {"max_depth": 6, "n_estimators": 25, "random_state": 27, "n_jobs": 8}

forest = RandomForestRegressor(**parameters)
forest.fit(data.drop([279], axis=1), data[279])

In [ ]:
n_top = 10
importances = forest.feature_importances_
idx = np.argsort(importances)[::-1][0:n_top]
feature_names = data.drop([279], axis=1).columns

plt.figure(figsize=(20, 5))
sns.barplot(x=feature_names[idx], y=importances[idx], palette="viridis")
plt.title("Топ 10 важных признаков", size=14)

In [ ]:
data['bmi'] = round(data[3] / ((data[2] / 100) ** 2), 1)
data.head(5)

In [ ]:
plt.figure(figsize=(16, 5))
sns.displot(data, x=data[0], bins=25 );

In [ ]:
plt.figure(figsize=(16, 5))
sns.kdeplot(data=data[0]);

In [ ]:
data['Age Sqrt'] = np.sqrt(data[0])
data

In [ ]:
data['Heart_rate_age'] = data[14] / data[0]
data

In [ ]:
x_data = data.drop([279], axis=1)
y_data = data[279]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [ ]:
model = xgb.XGBClassifier()

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

In [ ]:
def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0))
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0))
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0))
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0))
    return f1, roc, rec, prc

In [ ]:
f1, roc, rec, prc = evaluate_results(y_test, y_predict)

score = []
score.append([f1, prc, rec, roc])

разобраться с SHAP и построить важности признаков для:

- всего тестового набора данных (summary_plot - дать интерпретацию)

- для топ 10%

- для отдельных наблюдений вывести force_plot и попытаться проинтерпретировать результат

In [ ]:
# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(x_train)

# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
shap.force_plot(explainer.expected_value, shap_values[0,:], x_train.iloc[0,:])

In [ ]:
# summarize the effects of all the features
shap.summary_plot(shap_values, x_train, max_display=83)

Интерпретация summary_plot

индекс массы тела bmi показывает что его превышение увеличивает риск нарушений работы сердца. А его уменьшение, не снижает, а просто оставляет в районе нуля.

14-й признак (пульс) - увеличение пульса увеличивает риск нарушений работы сердца

heart_rate_age - отношение пульса к возрасту показывает, что повышенное или пониженное отношение понижает риски.

признак 3 (вес) ясно показывает что повышенный вес повышает, а пониженый - снижает риск.

In [ ]:
#  средние значения SHAP для каждого признака
shap.summary_plot(shap_values, x_train, plot_type="bar")

In [ ]:
# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
shap.force_plot(explainer.expected_value, shap_values[196,:], x_train.iloc[196,:], matplotlib=True)

In [ ]:
# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
shap.force_plot(explainer.expected_value, shap_values[176,:], x_train.iloc[176,:], matplotlib=True)

In [ ]:
# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
shap.force_plot(explainer.expected_value, shap_values[25,:], x_train.iloc[25,:], matplotlib=True)

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[20,:], x_train.iloc[20,:], matplotlib=True)

In [ ]:
# summary_plot для топ 10%
shap.summary_plot(shap_values, x_train, max_display=10)

In [ ]:
# summary_plot для топ 10% в виде баров
shap.summary_plot(shap_values, x_train, plot_type="bar", max_display=10)